In [16]:
!pip install -i https://pypi.org/simple/ bitsandbytes accelerate

Looking in indexes: https://pypi.org/simple/


In [17]:
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
from peft import PeftModel
from abc import ABC, abstractmethod

In [9]:
class TextGenerator(ABC):
    """
    Abstract base class for text generation models.
    """

    @abstractmethod
    def generate(self, prompt: str, max_length: int = 256) -> str:
        """
        Generate text based on the input prompt.
        
        Args:
        prompt (str): The input text prompt to generate text from.
        max_length (int): The maximum length of the generated text.
        
        Returns:
        str: The generated text.
        """
        pass

In [22]:
class FaunoModel(TextGenerator):
    def __init__(self, device: str = "cpu"):
        self.tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
        self.model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            #load_in_8bit=True,
            device_map=device
        )
        self.model = PeftModel.from_pretrained(self.model, "andreabac3/Open_Fauno-Italian-LLM-7bB")
        self.model.eval()

    def generate(self, question, max_length=256):
        prompt = f"The conversation between human and AI assistant.\n[|Human|] {question}.\n[|AI|] "
        inputs = self.tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"]
        generation_output = self.model.generate(
            input_ids=input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
        )
        output = self.tokenizer.decode(generation_output.sequences[0]).split("[|AI|]")[1]
        return output

In [11]:
class LLaMantinoModel(TextGenerator):
    def __init__(self,
                 device: str = "cpu", 
                 model_id: str = "swap-uniba/LLaMAntino-2-7b-hf-dolly-ITA",
                 quantization : str = "float8"
                ):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.tokenizer.add_special_tokens({"pad_token":"<unk>"})
        self.tokenizer.chat_template =   "{% set ns = namespace(i=0) %}" \
                                    "{% for message in messages %}" \
                                        "{% if message['role'] == 'user' and ns.i == 0 %}" \
                                               "{{ bos_token +' [INST] <<SYS>>\n' }}" \
                                               "{{ 'Sei un assistente disponibile, rispettoso e onesto di nome Llamantino. ' }}" \
                                               "{{ 'Rispondi sempre nel modo più utile possibile, pur essendo sicuro. ' }}" \
                                               "{{ 'Le risposte non devono includere contenuti dannosi, non etici, razzisti, sessisti, tossici, pericolosi o illegali. ' }}" \
                                               "{{ 'Assicurati che le tue risposte siano socialmente imparziali e positive. ' }}" \
                                               "{{ 'Se una domanda non ha senso o non è coerente con i fatti, spiegane il motivo invece di rispondere in modo non corretto. ' }}" \
                                               "{{ 'Se non conosci la risposta a una domanda, non condividere informazioni false.\n' }}" \
                                               "{{ '<</SYS>>\n\n' }}" \
                                               "{{ message['content'] + ' [/INST]' }}" \
                                        "{% elif message['role'] == 'user' and ns.i != 0 %} " \
                                            "{{ bos_token + ' [INST] ' + message['content'] + ' [/INST]' }}" \
                                        "{% elif message['role'] == 'assistant' %}" \
                                            "{{ ' '  + message['content'] + ' ' + eos_token + ' ' }}" \
                                        "{% endif %}" \
                                        "{% set ns.i = ns.i+1 %}" \
                                    "{% endfor %}"
        
        quantization_config = QuantoConfig(weights=quantization)
        self.model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device, quantization_config=quantization_config)
        self.model.eval()

        self.pipe = pipeline(model=self.model,
            device_map="balanced",
            tokenizer=self.tokenizer,
            return_full_text=False,  # langchain expects the full text
            task='text-generation',
            max_new_tokens=512,  # max number of tokens to generate in the output
            temperature=0.7 #temperature
        )

    def generate(self, prompt, max_length=256):
        messages = [{"role": "user", "content": prompt}]
        text = self.tokenizer.apply_chat_template(messages, tokenize=False)
        
        sequences = self.pipe(text)
        output = ""
        for seq in sequences:
            output += output + seq['generated_text']
        return output

## Data Loading

In [ ]:
triplet = pd.read_csv("wikidata5m_all_triplet.txt", sep="\t", names = ["Subject", "Predicate", "Object"])

In [ ]:
#entity = pd.read_csv("wikidata5m_entity.txt", sep='\t', usecols=[0, 1],  names = ["qID", "Name"], header=None)
#entity

# Text Generation

In [23]:
fauno7b = FaunoModel(device = "cpu")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [24]:
fauno7b.generate("Qual'è il significato della vita?")

' 1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [ ]:
llamantino7b = LLaMantinoModel()
llamantino7b.generate("Qual'è il significato della vita?")